#CNNs with Gated Convolutions

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Small.csv', index_col=False)
texts = df['Tweet']
labels = df['Sarcasm'].map({'yes': 1, 'no': 0})

In [3]:
def predict_sarcasm(text, tokenizer, model, max_len):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded_sequence)
    if prediction > 0.5:
        return "This text is predicted to be sarcastic."
    else:
        return "This text is predicted to be non-sarcastic."

In [4]:
max_len = 100
vocab_size = 10000
embedding_dim = 128
filter_sizes = [3, 4, 5]
num_filters = 64
texts = df['Tweet']
labels = df['Sarcasm']
labels = df['Sarcasm'].map({'yes': 1, 'no': 0})
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

padded_sequences = pad_sequences(sequences, maxlen=max_len)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

embedding_layer = Embedding(vocab_size, embedding_dim, input_length=max_len)

model = Sequential()
model.add(embedding_layer)

for filter_size in filter_sizes:
    conv_layer = Conv1D(num_filters, filter_size, activation='tanh', padding='same')
    gated_conv_layer = Conv1D(num_filters, filter_size, activation='sigmoid', padding='same')
    model.add(conv_layer)
    model.add(gated_conv_layer)
    model.add(Dropout(0.2))

model.add(GlobalMaxPooling1D())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

new_text = "My name is Tanisha."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

Epoch 1/10
174/174 [==============================] - 21s 104ms/step - loss: 0.6986 - accuracy: 0.5215 - val_loss: 0.6920 - val_accuracy: 0.5325
Epoch 2/10
174/174 [==============================] - 19s 108ms/step - loss: 0.5978 - accuracy: 0.6497 - val_loss: 0.3835 - val_accuracy: 0.8586
Epoch 3/10
174/174 [==============================] - 16s 90ms/step - loss: 0.2998 - accuracy: 0.8847 - val_loss: 0.3207 - val_accuracy: 0.8665
Epoch 4/10
174/174 [==============================] - 16s 93ms/step - loss: 0.1708 - accuracy: 0.9416 - val_loss: 0.2954 - val_accuracy: 0.8788
Epoch 5/10
174/174 [==============================] - 16s 90ms/step - loss: 0.1083 - accuracy: 0.9643 - val_loss: 0.3600 - val_accuracy: 0.8672
Epoch 6/10
174/174 [==============================] - 16s 93ms/step - loss: 0.0709 - accuracy: 0.9756 - val_loss: 0.3597 - val_accuracy: 0.8853
Epoch 7/10
174/174 [==============================] - 17s 96ms/step - loss: 0.0453 - accuracy: 0.9872 - val_loss: 0.3344 - val_accurac

In [5]:
new_text = "My mom asked me this question as well."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 21ms/step
This text is predicted to be non-sarcastic.


In [6]:
new_text = "Sure, let's just add this to my already overflowing to-do list."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 27ms/step
This text is predicted to be sarcastic.


In [7]:
new_text = "Sure, let's just add this to my already overflowing to-do list."
result = predict_sarcasm(new_text, tokenizer, model, max_len)
print(result)

1/1 [==============================] - 0s 21ms/step
This text is predicted to be sarcastic.
